In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv('data_cleaned.csv',encoding='latin1')

# Paso 1: Preparación de los Datos

Primero, necesitamos preparar nuestros datos. Vamos a crear una matriz de interacciones usuario-producto, donde cada fila representa un cliente y cada columna un producto. Los valores indicarán la cantidad comprada de cada producto por cada cliente.

In [16]:
from scipy.sparse import csr_matrix

# Crear una matriz de interacción usuario-producto.
# Agrupamos los datos por CustomerID y StockCode y sumamos la cantidad para cada combinación.
user_product_interaction = df.groupby(['CustomerID', 'StockCode'])['Quantity'].sum().unstack(fill_value=0)

# Convertimos esta matriz en una matriz dispersa (sparse) para mejorar la eficiencia en memoria y cálculo.
user_product_matrix = csr_matrix(user_product_interaction.values)

# Paso 2: Construcción del Modelo (Basico) de Recomendación con KNN

Usaremos el algoritmo Nearest Neighbors para encontrar productos similares basados en las compras de los clientes. Este modelo se entrenará con la matriz de interacciones para identificar similitudes entre productos.

In [17]:
from sklearn.neighbors import NearestNeighbors

# Inicializamos el modelo NearestNeighbors usando la métrica de similitud de coseno.
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

# Entrenamos el modelo con nuestra matriz de interacción producto-cliente.
# Usamos la transposición porque queremos la similitud entre productos, no entre clientes.
model_knn.fit(user_product_matrix.T)


NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

# Paso 3: Generación de Recomendaciones y Explicaciones

Para cada cliente, identificaremos productos que no ha comprado y encontraremos los más similares a los que sí ha comprado. Proporcionaremos una explicación basada en estos productos similares.

In [21]:
def get_recommendations(customer_id, n_recommendations=5):
    # Obtenemos los productos ya comprados por el cliente.
    purchased_indices = user_product_interaction.loc[customer_id].nonzero()[0]
    purchased_stockcodes = user_product_interaction.columns[purchased_indices]
    
    recommendations = []
    for stockcode in purchased_stockcodes:
        # Para cada producto comprado, encontramos productos similares.
        stockcode_idx = list(user_product_interaction.columns).index(stockcode)
        distances, indices = model_knn.kneighbors(user_product_interaction.iloc[:, stockcode_idx].values.reshape(1, -1), n_neighbors=n_recommendations+1)
        
        # Recorremos los productos similares encontrados.
        for i, (distance, index) in enumerate(zip(distances.flatten()[1:], indices.flatten()[1:]), start=1):
            similar_stockcode = user_product_interaction.columns[index]
            # Aseguramos que no recomendamos productos ya comprados.
            if similar_stockcode not in purchased_stockcodes:
                product_name = df[df['StockCode'] == similar_stockcode]['Description'].iloc[0]
                explanation = f"Porque compraste {df[df['StockCode'] == stockcode]['Description'].iloc[0]} (StockCode {stockcode}), que es similar."
                recommendations.append((similar_stockcode, product_name, explanation))
                if len(recommendations) == n_recommendations:
                    return recommendations
    return recommendations

# Paso 4: Aplicación del Proceso a Todos los Clientes y Exportación a Excel

Aplicaremos el proceso a una muestra de clientes y luego exportaremos los resultados a un archivo Excel.

In [23]:

n_recommendations = 5  # Ajusta este valor según el número deseado de recomendaciones por cliente

# Elegir una muestra de clientes para demostración
sample_customers = user_product_interaction.index[:100]  # Ajusta esto según tu capacidad de procesamiento

# Preparar contenedor para las recomendaciones de todos los clientes.
all_recommendations = []

# Iterar sobre un conjunto de clientes de ejemplo. Ajustar para iterar sobre todos los clientes según necesidades.
for customer_id in user_product_interaction.index[:100]:  # Ejemplo: los primeros 100 clientes.
    recs = get_recommendations(customer_id)
    for rec in recs:
        # Agregamos el ID del cliente y las recomendaciones (StockCode, Nombre, Explicación) a nuestra lista.
        all_recommendations.append([customer_id] + list(rec))

# Convertimos la lista de recomendaciones en un DataFrame.
recommendations_df = pd.DataFrame(all_recommendations, columns=['CustomerID', 'StockCode', 'ProductName', 'Explanation'])

# Exportamos el DataFrame a un archivo Excel.
recommendations_df.to_excel('customer_recommendations_with_explanations.xlsx', index=False)



'customer_recommendations_with_details.xlsx'

El sistema de recomendación que describí utiliza el algoritmo Nearest Neighbors basado en la similitud de coseno para identificar productos similares basándose en las interacciones de compra de los clientes. Aquí te explico cómo funciona y cómo se basa para recomendar

# Preparación de los datos: 

Primero, creamos una matriz de interacción usuario-producto, donde las filas representan a los clientes (CustomerID) y las columnas representan los productos (StockCode). Los valores de esta matriz indican la cantidad de cada producto comprada por cada cliente.

# Modelo de similitud de productos: 

Utilizamos el modelo Nearest Neighbors con la métrica de similitud de coseno para calcular qué tan similares son los productos entre sí, basándonos en todas las compras de los clientes. La similitud de coseno mide el ángulo entre dos vectores en el espacio multidimensional de la matriz de interacción, lo cual indica cuán similares son los patrones de compra de dos productos. Productos con patrones de compra similares tendrán una similitud de coseno más alta (más cercana a 1).

# Generación de recomendaciones: 

Para un cliente dado, el sistema identifica los productos que ya ha comprado y busca otros productos que sean similares a estos, basándose en la similitud de coseno calculada. Los productos recomendados son aquellos que son similares a los productos comprados por el cliente pero que el cliente aún no ha comprado.

Este proceso se basa en la suposición de que si un cliente compra un producto, es probable que esté interesado en otros productos similares en términos de cómo otros clientes interactúan con ellos. Por ejemplo, si muchos clientes compran los productos A y B juntos y un cliente específico solo ha comprado el producto A, el sistema puede recomendar el producto B a ese cliente, sugiriendo que productos comprados de manera similar por otros clientes también podrían ser de su interés.